# Movie ratings prediction

One of the most common uses of big data is to predict what users want.  This allows Google to show you relevant ads, Amazon to recommend relevant products, and Netflix to recommend movies that you might like.  This lab will demonstrate how you can use Apache Spark to recommend movies to a user.  We will start with some basic techniques, and then use the [Spark MLlib][http://spark.apache.org/mllib/] library's Alternating Least Squares method to make more sophisticated predictions.

For this lab, we will use a subset dataset of 500,000 ratings we included with th VM from the [movielens 10M stable benchmark rating dataset](http://grouplens.org/datasets/movielens/). However, the same code you write will work for the full dataset, or their latest dataset of 21 million ratings.

During the second part of the lab, you will submit a job using the full dataset on a Spark cluster deployed on Google Cloud.

Think carefully before calling `collect()` on any datasets.  When you are using a small dataset, calling `collect()` and then using Python to get a sense for the data locally (in the driver program) will work fine, but this will not work when you are using a large dataset that doesn't fit in memory on one machine.  Solutions that call `collect()` and do local analysis that could have been done without Spark will likely fail when running on a cluster.

In [2]:
#Please, run this first
import hashlib
import sys

def hash(x):
  return hashlib.sha1(str(x).encode('utf-8')).hexdigest()

assert sys.version_info.major == 3

In [3]:
import os

ratings_filename = '/FileStore/tables/ratings.dat'
movies_filename = '/FileStore/tables/movies.dat'

We read in each of the files and create an RDD consisting of parsed lines. Each line in the ratings dataset (`ratings.dat.gz`) is formatted as:
`user_id::movie_id::Rating::Timestamp`
Each line in the movies (`movies.dat`) dataset is formatted as:
`movie_id::Title::Genres`
The `Genres` field has the format
`Genres1|Genres2|Genres3|...`

The format of these files is uniform and simple, so you can use Python [`split()`](https://docs.python.org/2/library/stdtypes.html#str.split) to parse their lines.

Parsing the two files yields two RDDS:
* For each line in the ratings dataset, we create a tuple of (user_id, movie_id, Rating). We drop the timestamp because we do not need it for this exercise.
* For each line in the movies dataset, we create a tuple of (movie_id, Title). We drop the Genres because we do not need them for this exercise.

In [5]:
num_partitions = 2 # When running on a cluster, 
# you may want to increase this parameter according to the number of executors
raw_ratings = sc.textFile(ratings_filename).repartition(num_partitions)
raw_movies = sc.textFile(movies_filename)

def get_ratings_tuple(entry):
    """ Parse a line in the ratings dataset
    Args:
        entry (str): a line in the ratings dataset in the form of user_id::movie_id::Rating::Timestamp
    Returns:
        tuple: (user_id, movie_id, Rating)
    """
    items = entry.split('::')
    return int(items[0]), int(items[1]), float(items[2])


def get_movie_tuple(entry):
    """ Parse a line in the movies dataset
    Args:
        entry (str): a line in the movies dataset in the form of movie_id::Title::Genres
    Returns:
        tuple: (movie_id, Title)
    """
    items = entry.split('::')
    return int(items[0]), items[1]


ratings_RDD = raw_ratings.map(get_ratings_tuple).cache()
movies_RDD = raw_movies.map(get_movie_tuple).cache()

ratings_count = ratings_RDD.count()
movies_count = movies_RDD.count()

print('There are %s ratings and %s movies in the datasets' % (ratings_count, movies_count))
print('Ratings: %s' % ratings_RDD.take(3))
print('Movies: %s' % movies_RDD.take(3))

assert ratings_count == 487650
assert movies_count == 3883
assert movies_RDD.filter(lambda movie_tuple : movie_tuple[1] == 'Toy Story (1995)').count() == 1
assert (ratings_RDD.takeOrdered(1, key=lambda rating_tuple: rating_tuple[1]) == [(1, 1, 5.0)])

There are 487650 ratings and 3883 movies in the datasets
Ratings: [(1, 1193, 5.0), (1, 661, 3.0), (1, 914, 3.0)]
Movies: [(1, 'Toy Story (1995)'), (2, 'Jumanji (1995)'), (3, 'Grumpier Old Men (1995)')]

In this lab you will be creating and examining subsets of the tuples, such as the top rated movies by users.

Whenever you examine only a subset of a large dataset, there is the potential that the result will depend on the order you perform operations, such as joins, or how the data is partitioned across the workers. 

We want to guarantee that we always see the same results for a subset, independent of how we manipulate or store the data.

you can do that by sorting before examining a subset. You might think that the most obvious choice when dealing with an RDD of tuples would be to use the [`sortByKey()` method][sortbykey]. However this choice is problematic, as we can still end up with different results if the key is not unique.

**Note:** It is important to use the [`unicode` type](https://docs.python.org/2/howto/unicode.html#the-unicode-type) instead of the `string` type as the titles are in unicode characters.

Consider the following example, and note that while the sets are equal, the printed lists are usually in different order by value, *although they may randomly match up from time to time.*
You can try running this multiple times.  If the last assertion fails, don't worry about it: that was just the luck of the draw.  And note that in some environments, such as this virtual machine, the results are more deterministic than on a cluster with several workers.
[sortbykey]: https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortByKey

In [7]:
tmp1 = [(1, u'alpha'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'delta')]
tmp2 = [(1, u'delta'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'alpha')]

one_RDD = sc.parallelize(tmp1)
two_RDD = sc.parallelize(tmp2)
one_sorted = one_RDD.sortByKey(True).collect()
two_sorted = two_RDD.sortByKey(True).collect()
print(one_sorted)
print(two_sorted)
assert set(one_sorted) == set(two_sorted)     # Note that both lists have the same elements
assert two_sorted[0][0] < two_sorted.pop()[0] # Check that it is sorted by the keys
assert one_sorted[0:2] != two_sorted[0:2]     # Note that the subset consisting of the first two elements does not match

[(1, 'alpha'), (1, 'epsilon'), (1, 'delta'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]
[(1, 'delta'), (1, 'epsilon'), (1, 'alpha'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]

Even though the two lists contain identical tuples, the difference in ordering *sometimes* yields a different ordering for the sorted RDD. If we only examined the first two elements of the RDD (e.g., using `take(2)`), then we would observe different answers - **that is a really bad outcome as we want identical input data to always yield identical output**. 

A better technique is to sort the RDD by *both the key and value*, which we can do by combining the key and value into a single string and then sorting on that string. Since the key is an integer and the value is a unicode string, we can use a function to combine them into a single unicode string (e.g., `unicode('%.3f' % key) + ' ' + value`) before sorting the RDD using [sortBy()][sortby].
[sortby]: https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortBy

In [9]:
def sort_function(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, movie_name)
    Returns:
        sort_string: the value to sort with, 'rating movie_name'
    """
    key = str('%.3f' % tuple[0])
    value = tuple[1]
    return (key + ' ' + value)


print(one_RDD.sortBy(sort_function, True).collect())
print(two_RDD.sortBy(sort_function, True).collect())

[(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]
[(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]

If you just want to look at the first few elements of the RDD in sorted order, you can use the [takeOrdered][takeordered] method with the `sort_function` you defined.
[takeordered]: https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered

In [11]:
one_sorted1 = one_RDD.takeOrdered(one_RDD.count(),key=sort_function)
two_sorted1 = two_RDD.takeOrdered(two_RDD.count(),key=sort_function)
print('one is %s' % one_sorted1)
print('two is %s' % two_sorted1)
assert one_sorted1 == two_sorted1

one is [(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]
two is [(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]

One way to recommend movies is to always recommend the movies with the highest average rating. 

In this part, you will use Spark to find the name, number of ratings, and the average rating of the 20 movies with the highest average rating and more than 500 reviews. You want to filter out movies with high ratings but fewer than or equal to 500 reviews because movies with few reviews may not have broad appeal to everyone.


Using **only Python**, implement a helper function `get_counts_and_averages()` that takes a single tuple of (movie_id, (rating_1, rating_2, rating_3, ...)) and returns a tuple of (movie_id, (number of ratings, averageRating)). For example, given the tuple `(100, (10.0, 20.0, 30.0))`, your function should return `(100, (3, 20.0))`

In [13]:
def get_counts_and_averages(id_and_ratings_tuple):
    """ Calculate average rating
    Args:
        id_and_ratings_tuple: a single tuple of (movie_id, (rating_1, rating_2, rating_3, ...))
    Returns:
        tuple: a tuple of (movie_id, (number of ratings, averageRating))
    """
    number_of_ratings = len(id_and_ratings_tuple[1])
    averageRating = sum(id_and_ratings_tuple[1]) / number_of_ratings
    
    return (id_and_ratings_tuple[0], (number_of_ratings, averageRating))

  
assert get_counts_and_averages((1, (1, 2, 3, 4))) == (1, (4, 2.5)),\
                            'incorrect get_counts_and_averages() with integer list'
assert get_counts_and_averages((100, (10.0, 20.0, 30.0))) == (100, (3, 20.0)),\
                            'incorrect get_counts_and_averages() with float list'
assert get_counts_and_averages((110, range(20))) == (110, (20, 9.5)),\
                            'incorrect get_counts_and_averages() with range'

Now that we have a way to calculate the average ratings, we will use the `get_counts_and_averages()` helper function with Spark to determine movies with highest average ratings.
The steps you should perform are:
* Recall that the `ratings_RDD` contains tuples of the form (user_id, movie_id, Rating). From `ratings_RDD` create an RDD with tuples of the form (movie_id, Python iterable of Ratings for that movie_id). This transformation will yield an RDD of the form: `[(1, <pyspark.resultiterable.ResultIterable object at 0x7f16d50e7c90>), (2, <pyspark.resultiterable.ResultIterable object at 0x7f16d50e79d0>), (3, <pyspark.resultiterable.ResultIterable object at 0x7f16d50e7610>)]`. Note that you will only need to perform two Spark transformations to do this step.
* Using `movie_ids_with_ratings_RDD` and your `get_counts_and_averages()` helper function, compute the number of ratings and average rating for each movie to yield tuples of the form (movie_id, (number of ratings, average rating)). This transformation will yield an RDD of the form: `[(1, (993, 4.145015105740181)), (2, (332, 3.174698795180723)), (3, (299, 3.0468227424749164))]`. You can do this step with one Spark transformation
* We want to see movie names, instead of movie IDs. To `movies_RDD`, apply RDD transformations that use `movie_ids_with_avg_ratings_RDD` to get the movie names for `movie_ids_with_avg_ratings_RDD`, yielding tuples of the form (average rating, movie name, number of ratings). This set of transformations will yield an RDD of the form: `[(1.0, u'Autopsy (Macchie Solari) (1975)', 1), (1.0, u'Better Living (1998)', 1), (1.0, u'Big Squeeze, The (1996)', 3)]`. You will need to do two Spark transformations to complete this step: first use the `movies_RDD` with `movie_ids_with_avg_ratings_RDD` to create a new RDD with Movie names matched to Movie IDs, then convert that RDD into the form of (average rating, movie name, number of ratings). These transformations will yield an RDD that looks like: `[(3.6818181818181817, u'Happiest Millionaire, The (1967)', 22), (3.0468227424749164, u'Grumpier Old Men (1995)', 299), (2.882978723404255, u'Hocus Pocus (1993)', 94)]`

In [15]:
# From ratings_RDD with tuples of (user_id, movie_id, Rating) create an RDD with tuples of
# the (movie_id, iterable of Ratings for that movie_id)
movie_ids_with_ratings_RDD = (ratings_RDD
                              .map(lambda x: (x[1], x[2])).groupByKey())
print('movie_ids_with_ratings_RDD: %s\n' % movie_ids_with_ratings_RDD.take(3))

# Using `movie_ids_with_ratings_RDD`, compute the number of ratings and average rating for each movie to
# yield tuples of the form (movie_id, (number of ratings, average rating))
movie_ids_with_avg_ratings_RDD = movie_ids_with_ratings_RDD.map(get_counts_and_averages)
print('movie_ids_with_avg_ratings_RDD: %s\n' % movie_ids_with_avg_ratings_RDD.take(3))

# To `movie_ids_with_avg_ratings_RDD`, apply RDD transformations that use `movies_RDD` to get the movie
# names for `movie_ids_with_avg_ratings_RDD`, yielding tuples of the form
# (average rating, movie name, number of ratings)
movie_name_with_avg_ratings_RDD = movie_ids_with_avg_ratings_RDD.join(movies_RDD).map(lambda x : (x[1][0][1], x[1][1], x[1][0][0]))
print('movie_name_with_avg_ratings_RDD: %s\n' % movie_name_with_avg_ratings_RDD.take(3))

movie_ids_with_ratings_RDD: [(914, <pyspark.resultiterable.ResultIterable object at 0x7f6734064978>), (3408, <pyspark.resultiterable.ResultIterable object at 0x7f6734064da0>), (2804, <pyspark.resultiterable.ResultIterable object at 0x7f6734064f60>)]

movie_ids_with_avg_ratings_RDD: [(914, (314, 4.156050955414012)), (3408, (735, 3.8190476190476192)), (2804, (662, 4.2250755287009065))]

movie_name_with_avg_ratings_RDD: [(3.8190476190476192, 'Erin Brockovich (2000)', 735), (4.2250755287009065, 'Christmas Story, A (1983)', 662), (4.465346534653466, 'Wallace & Gromit: The Best of Aardman Animation (1996)', 202)]

In [16]:
# test the previous cell
assert movie_ids_with_ratings_RDD.count() == 3615,\
                'incorrect movie_ids_with_ratings_RDD.count() (expected 3615)'

movie_ids_with_ratings_take_ordered = movie_ids_with_ratings_RDD.takeOrdered(3)

assert (movie_ids_with_ratings_take_ordered[0][0] == 1 and
        len(list(movie_ids_with_ratings_take_ordered[0][1])) == 993),\
                'incorrect count of ratings for movie_ids_with_ratings_take_ordered[0] (expected 993)'

assert (movie_ids_with_ratings_take_ordered[1][0] == 2 and
                len(list(movie_ids_with_ratings_take_ordered[1][1])) == 332),\
                'incorrect count of ratings for movie_ids_with_ratings_take_ordered[1] (expected 332)'

assert (movie_ids_with_ratings_take_ordered[2][0] == 3 and
                len(list(movie_ids_with_ratings_take_ordered[2][1])) == 299),\
                'incorrect count of ratings for movie_ids_with_ratings_take_ordered[2] (expected 299)'

assert movie_ids_with_avg_ratings_RDD.count() == 3615,\
                'incorrect movie_ids_with_avg_ratings_RDD.count() (expected 3615)'

assert movie_ids_with_avg_ratings_RDD.takeOrdered(3) == \
                [(1, (993, 4.145015105740181)), (2, (332, 3.174698795180723)),
                 (3, (299, 3.0468227424749164))],\
                'incorrect movie_ids_with_avg_ratings_RDD.takeOrdered(3)'

assert movie_name_with_avg_ratings_RDD.count() == 3615,\
                'incorrect movie_name_with_avg_ratings_RDD.count() (expected 3615)'

assert movie_name_with_avg_ratings_RDD.takeOrdered(3) == \
                [(1.0, u'Autopsy (Macchie Solari) (1975)', 1), (1.0, u'Better Living (1998)', 1),
                 (1.0, u'Big Squeeze, The (1996)', 3)],\
                 'incorrect movie_name_with_avg_ratings_RDD.takeOrdered(3)'

Now that we have an RDD of the movies with highest averge ratings, we can use Spark to determine the 20 movies with highest average ratings and more than 500 reviews.

Apply a single RDD transformation to `movie_name_with_avg_ratings_RDD` to limit the results to movies with ratings from more than 500 people. Then, use the `sort_function()` helper function to sort by the average rating to get the movies in order of their rating (highest rating first). You will end up with an RDD of the form: `[(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088), (4.515798462852263, u"Schindler's List (1993)", 1171), (4.512893982808023, u'Godfather, The (1972)', 1047)]`

In [18]:
# Apply an RDD transformation to `movie_name_with_avg_ratings_RDD` to limit the results to movies with
# ratings from more than 500 people. We then use the `sort_function()` helper function to sort by the
# average rating to get the movies in order of their rating (highest rating first)
movie_limited_and_sorted_by_rating_RDD = (movie_name_with_avg_ratings_RDD
                                    .filter(lambda x : x[2] > 500)
                                    .sortBy(sort_function, False))
print('Movies with highest ratings: %s' % movie_limited_and_sorted_by_rating_RDD.take(20))

Movies with highest ratings: [(4.5349264705882355, 'Shawshank Redemption, The (1994)', 1088), (4.515798462852263, "Schindler's List (1993)", 1171), (4.512893982808023, 'Godfather, The (1972)', 1047), (4.510460251046025, 'Raiders of the Lost Ark (1981)', 1195), (4.505415162454874, 'Usual Suspects, The (1995)', 831), (4.457256461232604, 'Rear Window (1954)', 503), (4.45468509984639, 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651), (4.43953006219765, 'Star Wars: Episode IV - A New Hope (1977)', 1447), (4.4, 'Sixth Sense, The (1999)', 1110), (4.394285714285714, 'North by Northwest (1959)', 700), (4.379506641366224, 'Citizen Kane (1941)', 527), (4.375, 'Casablanca (1942)', 776), (4.363975155279503, 'Godfather: Part II, The (1974)', 805), (4.358816276202219, "One Flew Over the Cuckoo's Nest (1975)", 811), (4.358173076923077, 'Silence of the Lambs, The (1991)', 1248), (4.335826477187734, 'Saving Private Ryan (1998)', 1337), (4.326241134751773, 'Chinatown (1974)', 564), (4.325383304940375, 'Life Is Beautiful (La Vita � bella) (1997)', 587), (4.324110671936759, 'Monty Python and the Holy Grail (1974)', 759), (4.3096, 'Matrix, The (1999)', 1250)]

In [19]:
assert movie_limited_and_sorted_by_rating_RDD.count() == 194,\
                'incorrect movie_limited_and_sorted_by_rating_RDD.count()'

assert movie_limited_and_sorted_by_rating_RDD.take(20) == \
              [(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088),
               (4.515798462852263, u"Schindler's List (1993)", 1171),
               (4.512893982808023, u'Godfather, The (1972)', 1047),
               (4.510460251046025, u'Raiders of the Lost Ark (1981)', 1195),
               (4.505415162454874, u'Usual Suspects, The (1995)', 831),
               (4.457256461232604, u'Rear Window (1954)', 503),
               (4.45468509984639, u'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651),
               (4.43953006219765, u'Star Wars: Episode IV - A New Hope (1977)', 1447),
               (4.4, u'Sixth Sense, The (1999)', 1110), (4.394285714285714, u'North by Northwest (1959)', 700),
               (4.379506641366224, u'Citizen Kane (1941)', 527), (4.375, u'Casablanca (1942)', 776),
               (4.363975155279503, u'Godfather: Part II, The (1974)', 805),
               (4.358816276202219, u"One Flew Over the Cuckoo's Nest (1975)", 811),
               (4.358173076923077, u'Silence of the Lambs, The (1991)', 1248),
               (4.335826477187734, u'Saving Private Ryan (1998)', 1337),
               (4.326241134751773, u'Chinatown (1974)', 564),
               (4.325383304940375, u'Life Is Beautiful (La Vita \ufffd bella) (1997)', 587),
               (4.324110671936759, u'Monty Python and the Holy Grail (1974)', 759),
               (4.3096, u'Matrix, The (1999)', 1250)],\
        'incorrect sortedByRating_RDD.take(20)'

Using a threshold on the number of reviews is one way to improve the recommendations, but there are many other good ways to improve quality. For example, you could weight ratings by the number of ratings.

In this practical, you will learn how to use MLlib to make personalized movie recommendations using the movie data we have been analyzing.

You are going to use a technique called [collaborative filtering][collab]. Collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a person chosen randomly. You can read more about collaborative filtering [here][collab2].

The image below (from [Wikipedia][collab]) shows an example of predicting of the user's rating using collaborative filtering. At first, people rate different items (like videos, images, games). After that, the system is making predictions about a user's rating for an item, which the user has not rated yet. These predictions are built upon the existing ratings of other users, who have similar ratings with the active user. For instance, in the image below the system has made a prediction, that the active user will not like the video.
![collaborative filtering](https://courses.edx.org/c4x/BerkeleyX/CS100.1x/asset/Collaborative_filtering.gif)
[mllib]: https://spark.apache.org/mllib/
[collab]: https://en.wikipedia.org/?title=Collaborative_filtering
[collab2]: http://recommender-systems.org/collaborative-filtering/

For movie recommendations, we start with a matrix whose entries are movie ratings by users (shown in red in the diagram below).  Each column represents a user (shown in green) and each row represents a particular movie (shown in blue).

Since not all users have rated all movies, we do not know all of the entries in this matrix, which is precisely why we need collaborative filtering.  For each user, we have ratings for only a subset of the movies.  With collaborative filtering, the idea is to approximate the ratings matrix by factorizing it as the product of two matrices: one that describes properties of each user (shown in green), and one that describes properties of each movie (shown in blue).
![factorization](http://spark-mooc.github.io/web-assets/images/matrix_factorization.png)

We want to select these two matrices such that the error for the users/movie pairs where we know the correct ratings is minimized.  The [Alternating Least Squares][als] algorithm does this by first randomly filling the users matrix with values and then optimizing the value of the movies such that the error is minimized.  Then, it holds the movies matrix constrant and optimizes the value of the user's matrix.  This alternation between which matrix to optimize is the reason for the "alternating" in the name.

This optimization is what's being shown on the right in the image above.  Given a fixed set of user factors (i.e., values in the users matrix), we use the known ratings to find the best values for the movie factors using the optimization written at the bottom of the figure.  Then we "alternate" and pick the best user factors given fixed movie factors.

[als]: https://en.wikiversity.org/wiki/Least-Squares_Method

Before you jump into using machine learning, we need to break up the `ratings_RDD` dataset into three pieces:
* A training set (RDD), which we will use to train models
* A validation set (RDD), which we will use to choose the best model
* A test set (RDD), which we will use for our experiments

To randomly split the dataset into the multiple groups, we can use the pySpark [randomSplit()](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) transformation. `randomSplit()` takes a set of splits and and seed and returns multiple RDDs.

In [22]:
training_RDD, validation_RDD, test_RDD = ratings_RDD.randomSplit([6, 2, 2], seed=0)

print('Training: %s, validation: %s, test: %s\n' % (training_RDD.count(),
                                                    validation_RDD.count(),
                                                    test_RDD.count()))
print(training_RDD.take(3))
print(validation_RDD.take(3))
print(test_RDD.take(3))

assert training_RDD.count() == 293180
assert validation_RDD.count() == 96898
assert test_RDD.count() == 97572

assert training_RDD.filter(lambda t: t == (1, 1193, 5.0)).count() == 1
assert training_RDD.filter(lambda t: t == (1, 661, 3.0)).count() == 1
assert training_RDD.filter(lambda t: t == (1, 2355, 5.0)).count() == 1

assert validation_RDD.filter(lambda t: t == (1, 3408, 4.0)).count() == 1
assert validation_RDD.filter(lambda t: t == (1, 914, 3.0)).count() == 1
assert validation_RDD.filter(lambda t: t == (1, 2321, 3.0)).count() == 1

assert test_RDD.filter(lambda t: t == (1, 1197, 3.0)).count() == 1
assert test_RDD.filter(lambda t: t == (1, 1287, 5.0)).count() == 1
assert test_RDD.filter(lambda t: t == (1, 2804, 5.0)).count() == 1

Training: 293180, validation: 96898, test: 97572

[(1, 1193, 5.0), (1, 661, 3.0), (1, 2355, 5.0)]
[(1, 914, 3.0), (1, 3408, 4.0), (1, 2321, 3.0)]
[(1, 1197, 3.0), (1, 1287, 5.0), (1, 2804, 5.0)]

After splitting the dataset, your training set has about 293,000 entries and the validation and test sets each have about 97,000 entries (the exact number of entries in each dataset varies slightly due to the random nature of the `randomSplit()` transformation.

In the next part, you will generate a few different models, and will need a way to decide which model is best. We will use the [Root Mean Square Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE) or Root Mean Square Deviation (RMSD) to compute the error of each model. 

RMSE is a frequently used measure of the differences between values (sample and population values) predicted by a model or an estimator and the values actually observed. The RMSD represents the sample standard deviation of the differences between predicted values and observed values. These individual differences are called residuals when the calculations are performed over the data sample that was used for estimation, and are called prediction errors when computed out-of-sample. The RMSE serves to aggregate the magnitudes of the errors in predictions for various times into a single measure of predictive power. RMSE is a good measure of accuracy, but only to compare forecasting errors of different models for a particular variable and not between variables, as it is scale-dependent.

 The RMSE is the square root of the average value of the square of `(actual rating - predicted rating)` for all users and movies for which we have the actual rating. Versions of Spark MLlib beginning with Spark 1.4 include a [RegressionMetrics](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RegressionMetrics) module that can be used to compute the RMSE. However, you are here to learn, you will write your own function :)
 
Write a function to compute the sum of squared error given `predicted_RDD` and `actual_RDD` RDDs. Both RDDs consist of tuples of the form (user_id, movie_id, Rating)

Given two ratings RDDs, *x* and *y* of size *n*, we define RSME as follows: $ RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}$

To calculate RSME, the steps you should perform are:
* Transform `predicted_RDD` into the tuples of the form ((user_id, movie_id), Rating). For example, tuples like `[((1, 1), 5), ((1, 2), 3), ((1, 3), 4), ((2, 1), 3), ((2, 2), 2), ((2, 3), 4)]`. You can perform this step with a single Spark transformation.
* Transform `actual_RDD` into the tuples of the form ((user_id, movie_id), Rating). For example, tuples like `[((1, 2), 3), ((1, 3), 5), ((2, 1), 5), ((2, 2), 1)]`. You can perform this step with a single Spark transformation.
* Using only RDD transformations (you only need to perform two transformations), compute the squared error for each *matching* entry (i.e., the same (user_id, movie_id) in each RDD) in the reformatted RDDs - do *not* use `collect()` to perform this step. Note that not every (user_id, movie_id) pair will appear in both RDDs - if a pair does not appear in both RDDs, then it does not contribute to the RMSE. You will end up with an RDD with entries of the form $ (x_i - y_i)^2$ You might want to check out Python's [math](https://docs.python.org/2/library/math.html) module to see how to compute these values
* Using an RDD action (but **not** `collect()`), compute the total squared error: $ SE = \sum_{i = 1}^{n} (x_i - y_i)^2 $
* Compute *n* by using an RDD action (but **not** `collect()`), to count the number of pairs for which you computed the total squared error
* Using the total squared error and the number of pairs, compute the RSME. Make sure you compute this value as a [float](https://docs.python.org/2/library/stdtypes.html#numeric-types-int-float-long-complex).

Note: Your solution must only use transformations and actions on RDDs. Do _not_ call `collect()` on either RDD.

In [24]:
import math

def compute_error(predicted_RDD, actual_RDD):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predicted_RDD: predicted ratings for each movie and each user where each entry is in the form
                      (user_id, movie_id, Rating)
        actual_RDD: actual ratings where each entry is in the form (user_id, movie_id, Rating)
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predicted_RDD into the tuples of the form ((user_id, movie_id), Rating)
    predicted_reformatted_RDD = predicted_RDD.map(lambda x : ((x[0], x[1]), x[2]))

    # Transform actual_RDD into the tuples of the form ((user_id, movie_id), Rating)
    actual_reformatted_RDD = actual_RDD.map(lambda x : ((x[0], x[1]), x[2]))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Movie ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squared_errors_RDD = (predicted_reformatted_RDD
                          .join(actual_reformatted_RDD)
                          .map(lambda x : math.pow((x[1][1] - x[1][0]),2))
                         )

    # Compute the total squared error - do not use collect()
    total_error = squared_errors_RDD.sum()

    # Count the number of entries for which you computed the total squared error
    num_ratings = squared_errors_RDD.count()

    # Using the total squared error and the number of entries, compute the RSME
    return math.sqrt(total_error/num_ratings)


# sc.parallelize turns a Python list into a Spark RDD.
test_predicted = sc.parallelize([
    (1, 1, 5),
    (1, 2, 3),
    (1, 3, 4),
    (2, 1, 3),
    (2, 2, 2),
    (2, 3, 4)])
test_actual = sc.parallelize([
     (1, 2, 3),
     (1, 3, 5),
     (2, 1, 5),
     (2, 2, 1)])
test_predicted2 = sc.parallelize([
     (2, 2, 5),
     (1, 2, 5)])
test_error = compute_error(test_predicted, test_actual)
print('RMSE for test dataset (should be 1.22474487139): %s' % test_error)

test_error2 = compute_error(test_predicted2, test_actual)
print('RMSE for test dataset2 (should be 3.16227766017): %s' % test_error2)

test_error3 = compute_error(test_actual, test_actual)
print('RMSE for test_actual dataset (should be 0.0): %s' % test_error3)

assert abs(test_error - 1.22474487139) < 0.00000001,\
                'incorrect test_error (expected 1.22474487139)'
assert abs(test_error2 - 3.16227766017) < 0.00000001,\
                'incorrect test_error2 result (expected 3.16227766017)'
assert abs(test_error3 - 0.0) < 0.00000001,\
                'incorrect test_actual result (expected 0.0)'

RMSE for test dataset (should be 1.22474487139): 1.224744871391589
RMSE for test dataset2 (should be 3.16227766017): 3.1622776601683795
RMSE for test_actual dataset (should be 0.0): 0.0

You will now use the MLlib implementation of Alternating Least Squares, [ALS.train()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS). ALS takes a training dataset (RDD) and several parameters that control the model creation process. To determine the best values for the parameters, you will use ALS to train several models, and then you will select the best model and use the parameters from that model in the rest of this lab exercise.

The process you will use for determining the best model is as follows:
* Pick a set of model parameters. The most important parameter to `ALS.train()` is the *rank*, which is the number of rows in the Users matrix (green in the diagram above) or the number of columns in the Movies matrix (blue in the diagram above). (In general, a lower rank will mean higher error on the training dataset, but a high rank may lead to [overfitting](https://en.wikipedia.org/wiki/Overfitting).)  You will train models with ranks of 4, 8, and 12 using the `training_RDD` dataset.
* Create a model using `ALS.train(training_RDD, rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)` with three parameters: an RDD consisting of tuples of the form (user_id, movie_id, rating) used to train the model, an integer rank (4, 8, or 12), a number of iterations to execute (we will use 5 for the `iterations` parameter), and a regularization coefficient (we will use 0.1 for the `regularization_parameter`).
* For the prediction step, create an input RDD, `validation_for_predict_RDD`, consisting of (user_id, movie_id) pairs that you extract from `validation_RDD`. You will end up with an RDD of the form: `[(1, 1287), (1, 594), (1, 1270)]`
* Using the model and `validation_for_predict_RDD`, you can predict rating values by calling [model.predictAll()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.predictAll) with the `validation_for_predict_RDD` dataset, where `model` is the model we generated with ALS.train().  `predictAll` accepts an RDD with each entry in the format (user_id, movie_id) and outputs an RDD with each entry in the format (user_id, movie_id, rating).
* Evaluate the quality of the model by using the `compute_error()` function you wrote ealier to compute the error between the predicted ratings and the actual ratings in `validation_RDD`.

Note: It is likely that this operation will take a noticeable amount of time (around a minute in our VM depending on your hardware); you can observe its progress on the [Spark Web UI](http://localhost:4040). Probably most of the time will be spent running your `compute_error()` function, since, unlike the Spark ALS implementation (and the Spark 1.4 [RegressionMetrics](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RegressionMetrics) module), this does not use a fast linear algebra library and needs to run some Python code for all 100k entries.

In [26]:
from pyspark.mllib.recommendation import ALS

validation_for_predict_RDD = validation_RDD.map(lambda x : (x[0],x[1]))

seed = 5
iterations = 5
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.03

min_error = float('inf')
best_rank = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predicted_ratings_RDD = model.predictAll(validation_for_predict_RDD)
    error = compute_error(predicted_ratings_RDD, validation_RDD)
    errors[err] = error
    err += 1
    print('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.8960745588092996
For rank 8 the RMSE is 0.8949686142293647
For rank 12 the RMSE is 0.891538951438127
The best model was trained with rank 12

In [27]:
assert training_RDD.getNumPartitions() == 2,\
                  'incorrect number of partitions for training_RDD (expected 2)'
assert validation_for_predict_RDD.count() == 96898,\
                  'incorrect size for validation_for_predict_RDD (expected 96898)'
assert validation_for_predict_RDD.filter(lambda t: t == (1, 3408)).count() == 1,\
                  'incorrect content for validation_for_predict_RDD'
assert abs(errors[0] - 0.895405660311) < tolerance, 'incorrect errors[0]'
assert abs(errors[1] - 0.895514822303) < tolerance, 'incorrect errors[1]'
assert abs(errors[2] - 0.894980442967) < tolerance, 'incorrect errors[2]'

So far, you used the `training_RDD` and `validation_RDD` datasets to select the best model.  Since we used these two datasets to determine what model is best, we cannot use them to test how good the model is - otherwise we would be very vulnerable to [overfitting](https://en.wikipedia.org/wiki/Overfitting).  To decide how good our model is, we need to use the `test_RDD` dataset.  We will use the `best_rank` you determined in part earlier to create a model for predicting the ratings for the test dataset and then we will compute the RMSE.

You should perform the following steps:
* Train a model, using the `training_RDD`, `best_rank` and the parameters you used from earlier: `seed=seed`, `iterations=iterations`, and `lambda_=regularization_parameter` - make sure you include **all** of the parameters.
* For the prediction step, create an input RDD, `test_for_predicting_RDD`, consisting of (user_id, movie_id) pairs that you extract from `test_RDD`. You will end up with an RDD of the form: `[(1, 1287), (1, 594), (1, 1270)]`
* Use [my_model.predictAll()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.predictAll) to predict rating values for the test dataset.
* For validation, use the `test_RDD`and your `compute_error` function to compute the RMSE between `test_RDD` and the `predicted_test_RDD` from the model.
* Evaluate the quality of the model by using the `compute_error()` function you wrote in part (2b) to compute the error between the predicted ratings and the actual ratings in `test_RDD`.

In [29]:
my_model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                     lambda_=regularization_parameter)
test_for_predicting_RDD = test_RDD.map(lambda x : (x[0],x[1]))
predicted_test_RDD = my_model.predictAll(test_for_predicting_RDD)

test_RMSE = compute_error(test_RDD, predicted_test_RDD)

print('The model had a RMSE on the test set of %s' % test_RMSE)

assert abs(test_RMSE - 0.896040796967) < tolerance, 'incorrect test_RMSE'

The model had a RMSE on the test set of 0.8933944907915308

Looking at the RMSE for the results predicted by the model versus the values in the test set is one way to evalute the quality of our model. Another way to evaluate the model is to evaluate the error from a test set where every rating is the average rating for the training set.

You should perform the following steps:
* Use the `training_RDD` to compute the average rating across all movies in that training dataset.
* Use the average rating that you just determined and the `test_RDD` to create an RDD with entries of the form (user_id, movie_id, average rating).
* Use your `compute_error` function to compute the RMSE between the `test_RDD` validation RDD that you just created and the `test_for_avg_RDD`.

In [31]:
training_avg_rating = training_RDD.map(lambda x : x[2]).mean()
print('The average rating for movies in the training set is %s' % training_avg_rating)

test_for_avg_RDD = test_RDD.map(lambda x : (x[0], x[1], training_avg_rating))
test_avg_RMSE = compute_error(test_RDD, test_for_avg_RDD)
print('The RMSE on the average set is %s' % test_avg_RMSE)

assert abs(training_avg_rating - 3.5716010641) < 0.000001,\
                'incorrect training_avg_rating (expected 3.5716010641)'
assert abs(test_avg_RMSE - 1.11441205015) < 0.000001,\
                'incorrect test_avg_RMSE (expected 1.11441205015)'

The average rating for movies in the training set is 3.571601064192641
The RMSE on the average set is 1.1144120501597654

You now have code to predict how users will rate movies!

The ultimate goal of this exercise is to predict what movies to recommend to yourself. In order to do that, you will first need to add ratings for yourself to the `ratings_RDD` dataset.

To help you provide ratings for yourself, use the following code cell to list the names and movie IDs of the 50 highest-rated movies from `movie_limited_and_sorted_by_rating_RDD` which we created in part 1 the lab.

In [33]:
print('Most rated movies:')
print('(average rating, movie name, number of reviews)')
for ratings_tuple in movie_limited_and_sorted_by_rating_RDD.take(50):
    print(ratings_tuple)

Most rated movies:
(average rating, movie name, number of reviews)
(4.5349264705882355, 'Shawshank Redemption, The (1994)', 1088)
(4.515798462852263, "Schindler's List (1993)", 1171)
(4.512893982808023, 'Godfather, The (1972)', 1047)
(4.510460251046025, 'Raiders of the Lost Ark (1981)', 1195)
(4.505415162454874, 'Usual Suspects, The (1995)', 831)
(4.457256461232604, 'Rear Window (1954)', 503)
(4.45468509984639, 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651)
(4.43953006219765, 'Star Wars: Episode IV - A New Hope (1977)', 1447)
(4.4, 'Sixth Sense, The (1999)', 1110)
(4.394285714285714, 'North by Northwest (1959)', 700)
(4.379506641366224, 'Citizen Kane (1941)', 527)
(4.375, 'Casablanca (1942)', 776)
(4.363975155279503, 'Godfather: Part II, The (1974)', 805)
(4.358816276202219, "One Flew Over the Cuckoo's Nest (1975)", 811)
(4.358173076923077, 'Silence of the Lambs, The (1991)', 1248)
(4.335826477187734, 'Saving Private Ryan (1998)', 1337)
(4.326241134751773, 'Chinatown (1974)', 564)
(4.325383304940375, 'Life Is Beautiful (La Vita � bella) (1997)', 587)
(4.324110671936759, 'Monty Python and the Holy Grail (1974)', 759)
(4.3096, 'Matrix, The (1999)', 1250)
(4.309457579972183, 'Star Wars: Episode V - The Empire Strikes Back (1980)', 1438)
(4.30379746835443, 'Young Frankenstein (1974)', 553)
(4.301346801346801, 'Psycho (1960)', 594)
(4.296438883541867, 'Pulp Fiction (1994)', 1039)
(4.286535303776683, 'Fargo (1996)', 1218)
(4.282367447595561, 'GoodFellas (1990)', 811)
(4.27943661971831, 'American Beauty (1999)', 1775)
(4.268053855569155, 'Wizard of Oz, The (1939)', 817)
(4.267774699907664, 'Princess Bride, The (1987)', 1083)
(4.253333333333333, 'Graduate, The (1967)', 600)
(4.236263736263736, 'Run Lola Run (Lola rennt) (1998)', 546)
(4.233807266982622, 'Amadeus (1984)', 633)
(4.232558139534884, 'Toy Story 2 (1999)', 860)
(4.232558139534884, 'This Is Spinal Tap (1984)', 516)
(4.228494623655914, 'Almost Famous (2000)', 744)
(4.2250755287009065, 'Christmas Story, A (1983)', 662)
(4.216757741347905, 'Glory (1989)', 549)
(4.213358070500927, 'Apocalypse Now (1979)', 539)
(4.20992028343667, 'L.A. Confidential (1997)', 1129)
(4.204733727810651, 'Blade Runner (1982)', 845)
(4.1886120996441285, 'Sling Blade (1996)', 562)
(4.184615384615385, 'Braveheart (1995)', 1300)
(4.184168012924071, 'Butch Cassidy and the Sundance Kid (1969)', 619)
(4.182509505703422, 'Good Will Hunting (1997)', 789)
(4.166969147005445, 'Taxi Driver (1976)', 551)
(4.162767039674466, 'Terminator, The (1984)', 983)
(4.157545605306799, 'Reservoir Dogs (1992)', 603)
(4.153333333333333, 'Jaws (1975)', 750)
(4.149840595111583, 'Alien (1979)', 941)
(4.145015105740181, 'Toy Story (1995)', 993)

The user ID 0 is unassigned, so you will use it for your ratings. Set the variable `myuser_id` to 0. 

Next, create a new RDD `my_ratings_RDD` with your ratings for at least 10 movie ratings. Each entry should be formatted as `(myuser_id, movie_id, rating)` (i.e., each entry should be formatted in the same way as `training_RDD`).  As in the original dataset, ratings should be between 1 and 5 (inclusive). If you have not seen at least 10 of these movies, you can increase the parameter passed to `take()` in the above cell until there are 10 movies that you have seen (or you can also guess what your rating would be for movies you have not seen).

In [35]:
myuser_id = 0

# Note that the movie IDs are the *last* number on each line.
my_rated_movies = [
     # The format of each line is (myuser_id, movie ID, your rating)
     # For example, to give the movie "Star Wars: Episode IV - A New Hope (1977)" 
     # a five rating, you would add the following line:
     #   (myuser_id, 260, 5),
  (myuser_id, 1447, 4),
  (myuser_id, 1039, 4),
  (myuser_id, 860, 3),
  (myuser_id, 587, 2),
  (myuser_id, 551, 4.5),
  (myuser_id, 1013, 2),
  (myuser_id, 1410, 4),
  (myuser_id, 830, 6),
  (myuser_id, 112, 4),
  (myuser_id, 247, 5),
  (myuser_id, 269, 5),
  (myuser_id, 600, 5),
  (myuser_id, 624, 5),
  (myuser_id, 700, 3)
    ]

my_ratings_RDD = sc.parallelize(my_rated_movies)
print('My movie ratings: %s' % my_ratings_RDD.take(10))

My movie ratings: [(0, 1447, 4), (0, 1039, 4), (0, 860, 3), (0, 587, 2), (0, 551, 4.5), (0, 1013, 2), (0, 1410, 4), (0, 830, 6), (0, 112, 4), (0, 247, 5)]

Now that you have ratings for yourself, you need to add your ratings to the `training` dataset so that the model you train will incorporate your preferences.  Spark's [union()](http://spark.apache.org/docs/latest/api/python/pyspark.rdd.RDD-class.html#union) transformation combines two RDDs; use `union()` to create a new training dataset that includes your ratings and the data in the original training dataset.

In [37]:
training_with_my_ratings_RDD = training_RDD.union(my_ratings_RDD)

print('The training dataset now has %s more entries than the original training dataset' %
       (training_with_my_ratings_RDD.count() - training_RDD.count()))
assert (training_with_my_ratings_RDD.count() - training_RDD.count()) == my_ratings_RDD.count()

The training dataset now has 14 more entries than the original training dataset

Now, train a model with your ratings added and the parameters you used in in part (2c): `best_rank`, `seed=seed`, `iterations=iterations`, and `lambda_=regularization_parameter` - make sure you include **all** of the parameters.

In [39]:
my_ratings_model = ALS.train(training_with_my_ratings_RDD, best_rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)

Compute the RMSE for this new model on the test set.
* For the prediction step, reuse `test_for_predicting_RDD`, consisting of (user_id, movie_id) pairs that you extracted from `test_RDD`. The RDD has the form: `[(1, 1287), (1, 594), (1, 1270)]`
* Use `my_ratings_model.predictAll()` to predict rating values for the `test_for_predicting_RDD` test dataset, set this as `predictedtest_my_ratings_RDD`
* For validation, use the `test_RDD`and your `compute_error` function to compute the RMSE between `test_RDD` and the `predictedtest_my_ratings_RDD` from the model.

In [41]:
predictedtest_my_ratings_RDD = my_ratings_model.predictAll(test_for_predicting_RDD)
test_RMSE_my_ratings = compute_error(test_RDD, predictedtest_my_ratings_RDD)
print('The model had a RMSE on the test set of %s' % test_RMSE_my_ratings)

The model had a RMSE on the test set of 0.8911440901764003

So far, we have only used the `predictAll` method to compute the error of the model.  Here, use the `predictAll` to predict what ratings you would give to the movies that you did not already provide ratings for.

You should perform the following steps:
* Use the Python list `my_rated_movies` to transform the `movies_RDD` into an RDD with entries that are pairs of the form (myuser_id, Movie ID) and that does not contain any movies that you have rated. This transformation will yield an RDD of the form: `[(0, 1), (0, 2), (0, 3), (0, 4)]`. Note that you can do this step with one RDD transformation.
* For the prediction step, use the input RDD, `my_unrated_movies_RDD`, with my_ratings_model.predictAll() to predict your ratings for the movies.

In [43]:
movies_RDD.take(5)

Out[23]: [(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (4, 'Waiting to Exhale (1995)'),
 (5, 'Father of the Bride Part II (1995)')]

In [44]:
# Use the Python list my_rated_movies to transform the movies_RDD into an RDD with entries that are 
# pairs of the form (myuser_id, Movie ID) and that does not contain any movies that you have rated.
my_rated_movies_id = [x[1] for x in my_rated_movies]
my_unrated_movies_RDD = (movies_RDD
                         .filter(lambda x : x[0] not in my_rated_movies_id)
                         .map(lambda x : (myuser_id, x[0]))
                         )

# Use the input RDD, my_unrated_movies_RDD, with my_ratings_model.predictAll() to predict your ratings for the movies
predicted_ratings_RDD = my_ratings_model.predictAll(my_unrated_movies_RDD)

You have our predicted ratings. Now you can print out the 25 movies with the highest predicted ratings.

You should perform the following steps:
* From earlier, you know that you should look at movies with a reasonable number of reviews (e.g., more than 75 reviews). You can experiment with a lower threshold, but fewer ratings for a movie may yield higher prediction errors. Transform `movie_ids_with_avg_ratings_RDD`, which has the form (movie_id, (number of ratings, average rating)), into an RDD of the form (movie_id, number of ratings): `[(2, 332), (4, 71), (6, 442)]`
* We want to see movie names, instead of movie IDs. Transform `predicted_ratings_RDD` into an RDD with entries that are pairs of the form (Movie ID, Predicted Rating): `[(3456, -0.5501005376936687), (1080, 1.5885892024487962), (320, -3.7952255522487865)]`
* Use RDD transformations with `predicted_RDD` and `movie_counts_RDD` to yield an RDD with tuples of the form (Movie ID, (Predicted Rating, number of ratings)): `[(2050, (0.6694097486155939, 44)), (10, (5.29762541533513, 418)), (2060, (0.5055259373841172, 97))]`
* Use RDD transformations with `predicted_with_counts_RDD` and `movies_RDD` to yield an RDD with tuples of the form (Predicted Rating, Movie Name, number of ratings), _for movies with more than 75 ratings._ For example: `[(7.983121900375243, u'Under Siege (1992)'), (7.9769201864261285, u'Fifth Element, The (1997)')]`

In [46]:
# Transform movie_ids_with_avg_ratings_RDD from part (1b), which has the form (movie_id, (number of ratings, average rating)),
# into and RDD of the form (movie_id, number of ratings)
movie_counts_RDD = movie_ids_with_avg_ratings_RDD.map(lambda x : (x[0], x[1][0]))

# Transform predicted_ratings_RDD into an RDD with entries that are pairs of the form (Movie ID, Predicted Rating)
predicted_RDD = predicted_ratings_RDD.map(lambda x : (x[1], x[2]))

# Use RDD transformations with predicted_RDD and movie_counts_RDD to yield an RDD with tuples of the form 
# (Movie ID, (Predicted Rating, number of ratings))
predicted_with_counts_RDD  = (predicted_RDD
                              .join(movie_counts_RDD)
                             )

# Use RDD transformations with Predicted_with_counts_RDD and movies_RDD to yield an RDD with tuples of the form
# (Predicted Rating, Movie Name, number of ratings), for movies with more than 75 ratings
ratings_with_names_RDD = (predicted_with_counts_RDD
                          .join(movies_RDD)
                          .map(lambda x : x[1])
                          .map(lambda x : (x[0][0], x[1], x[0][1]))
                         )

predicted_highest_rated_movies = ratings_with_names_RDD.takeOrdered(20, key=lambda x: -x[0])
print('My highest rated movies as predicted (for movies with more than 75 reviews):\n%s' %
        '\n'.join(map(str, predicted_highest_rated_movies)))

My highest rated movies as predicted (for movies with more than 75 reviews):
(6.685495597692892, 'Acid House, The (1998)', 4)
(5.9835895442673195, 'Arguing the World (1996)', 5)
(5.764762326684784, 'Eighth Day, The (Le Huiti�me jour ) (1996)', 5)
(5.726276710017781, 'Saltmen of Tibet, The (1997)', 6)
(5.578066472828304, 'Mamma Roma (1962)', 1)
(5.575520099875391, 'Crimson Pirate, The (1952)', 38)
(5.571524638262723, 'Slam (1998)', 12)
(5.534641023148072, 'Fanny and Alexander (1982)', 62)
(5.503655854855887, 'Metropolis (1926)', 177)
(5.4754543512734175, 'Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922)', 115)
(5.402483988992488, 'Character (Karakter) (1997)', 21)
(5.363314353059822, 'Of Human Bondage (1934)', 29)
(5.356737864862688, 'Firelight (1997)', 5)
(5.349979516463824, 'Land and Freedom (Tierra y libertad) (1995)', 6)
(5.337665404583797, 'Lodger, The (1926)', 11)
(5.32218735794252, 'Funhouse, The (1981)', 12)
(5.3172827521120265, 'Tarantula (1955)', 30)
(5.302139086520617, 'It Came from Outer Space (1953)', 46)
(5.236005317779915, 'Dersu Uzala (1974)', 45)
(5.234120331251189, 'Ballad of Narayama, The (Narayama Bushiko) (1982)', 8)

In [47]:
# You can save your model using the model.save()
model.save(sc, "./model_save.mfm")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1307889669389322> in <module> 
 1 # You can save your model using the model.save() 
 ----> 2 model . save ( sc , "./model_save.mfm" ) 

 /databricks/spark/python/pyspark/mllib/util.py in save (self, sc, path) 
 404 if not isinstance ( path , basestring ) : 
 405 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 406 self . _java_model . save ( sc . _jsc . sc ( ) , path ) 
 407 
 408 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1873.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory dbfs:/model_save.mfm/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:289)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:392)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply(PairRDDFunctions.scala:1032)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply(PairRDDFunctions.scala:1032)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:392)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopFile(PairRDDFunctions.scala:1032)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$1.apply$mcV$sp(PairRDDFunctions.scala:958)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$1.apply(PairRDDFunctions.scala:958)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$1.apply(PairRDDFunctions.scala:958)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:392)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopFile(PairRDDFunctions.scala:957)
	at org.apache.spark.rdd.RDD$$anonfun$saveAsTextFile$1.apply$mcV$sp(RDD.scala:1562)
	at org.apache.spark.rdd.RDD$$anonfun$saveAsTextFile$1.apply(RDD.scala:1541)
	at org.apache.spark.rdd.RDD$$anonfun$saveAsTextFile$1.apply(RDD.scala:1541)
	at org.

In [48]:
# And then, load it by doing:
from pyspark.mllib.recommendation import MatrixFactorizationModel
test = MatrixFactorizationModel.load(sc, './model_save.mfm')

# Why do we use MatrixFactorizationModel to load the model?
print(model)  # Because ALS models are an instance of the pyspark.mllib.recommendation.MatrixFactorizationModel class